# It's a Machine and Natural Language Model

In [1]:
from src.IaMaN.base import LM
from src.utils.data import load_wikitext
from src.utils.munge import stick_spaces
from collections import defaultdict
from collections import Counter
from tqdm import tqdm
import numpy as np
import os, re

seed = 691; max_char = 200_000_000
m = 10; space = True; fine_tune = False; num_posttrain = 0; noise = 0.001
positional = 'dependent'; positionally_encode = True; bits = 50; update = True; btype = 'df'; ms_init = 'waiting_time'
runners = 10; gpu = False; tokenizer = 'hr-bpe' # 'sentokenizer' #

total_posttrain = 0; ptdocs = []
if num_posttrain:
    num_posttrain = 0 if num_posttrain == 'all' else num_posttrain
    print("Loading WikiText-103 as post-training data...")
    posttrain_docs = load_wikitext(v = '103', split = "train", num_articles = num_posttrain, seed = seed, space = space, rebuild = True)
    ptdocs = [["".join(s) for s in d['document']] for d in posttrain_docs]; ptdocs = [d for d in ptdocs if len(d)]
print("Loading WikiText-2 LM'ing data...")
train_docs = load_wikitext(v = '2', split = "train", num_articles = 0, seed = seed, space = space, rebuild = True)
test_docs = load_wikitext(v = '2', split = "test", num_articles = 0, seed = seed, space = space, rebuild = True)
test_docs = [doc for doc in test_docs if len(doc['text']) and (len(doc['text']) <= max_char)]# [:2]
train_docs = [doc for doc in train_docs if len(doc['text']) and (len(doc['text']) <= max_char)]# [:4]
print('Avail. post-train, total post-train, Avail. gold, total gold-train, total test-gold: ', 
      total_posttrain, len(ptdocs), len(train_docs) + len(test_docs), len(train_docs), len(test_docs))

Loading WikiText-2 LM'ing data...
Avail. post-train, total post-train, Avail. gold, total gold-train, total test-gold:  0 0 672 611 61


In [2]:
docs = [["".join(s) for s in d['document']] for d in train_docs]
tdocs = [["".join(s) for s in d['document']] for d in test_docs]
covering = [[re.split('( )', ''.join(s)) for s in d['document']] for d in train_docs]
tcovering = [[re.split('( )', ''.join(s)) for s in d['document']] for d in test_docs]
if not space:
    for d_i, d in enumerate(covering):
        for s_i, s in enumerate(d):
            covering[d_i][s_i] = stick_spaces(s)
    for d_i, d in enumerate(tcovering):
        for s_i, s in enumerate(d):
            tcovering[d_i][s_i] = stick_spaces(s)            
covering_vocab = set([t for d in covering for s in d for t in s])
model = LM(m = m, tokenizer = tokenizer, noise = noise, seed = seed, space = space, positional = positional,
           positionally_encode = positionally_encode, bits = bits, runners = runners, gpu = gpu, 
           btype = btype, ms_init = ms_init)
data_streams = model.fit(docs, f'WT2-{len(test_docs)}', fine_tune = fine_tune, covering = covering)

Training tokenizer...


Fitting:   7%|▋         | 7/100 [02:20<31:05, 20.05s/it]


Built a vocabulary of 34914 types
Tokenizing documents...


100%|██████████| 611/611 [08:08<00:00,  1.25it/s]


Pre-processing data...


...

Building dense output heads...


100%|██████████| 15/15 [00:01<00:00, 11.13it/s]


Building transition matrices for tag-sequence decoding...


  0%|          | 0/6 [00:00<?, ?it/s]/code/IaMaN/src/IaMaN/base.py:613: RuntimeWarning: invalid value encountered in true_divide
  self._trXs[ltype] = (lambda M: M/M.sum(axis = 1)[:,None])(self._trXs[ltype])
100%|██████████| 6/6 [00:00<00:00, 8322.03it/s]

Done.
Model params, types, encoding size, contexts, vec dim, max sent, and % capacity used: 5080629 24042 50 1071 63 1647 31.874


In [3]:
model._ms

{'nov': 2,
 'iat': 2,
 'bot': 2,
 'eot': 2,
 'eos': 2,
 'eod': 2,
 'form': 6,
 'bits': 9}

In [4]:
model.generate(m = 50, seed = seed)

McCorduckGertrudeMcCorduckernick North , <unk> tongue whom . Public filmeum day , though a University in the downfall w the 1960s dropped performance the


```
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  1 2.0 27.74 236.2 1696 982
```

In [5]:
out = model.stencil(docs = tdocs, return_output = True, covering = tcovering)

Tokenizing documents...


100%|██████████| 61/61 [00:52<00:00,  1.16it/s]


Pre-processing data...


  2%|▏         | 1/61 [00:08<08:39,  8.66s/it]

document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  1 2.0 27.18 226.85 1696 982


...

document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  59 97.0 35.9 358.56 11172 6381


 98%|█████████▊| 60/61 [45:11<00:40, 40.22s/it]

document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  60 98.0 30.2 273.01 8388 4807


100%|██████████| 61/61 [45:52<00:00, 45.12s/it]

document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  61 100.0 40.85 440.31 7335 4164


In [6]:
print('micro: ', round(1/(10**(np.log10([p[0] for ps in out for p in ps]).mean())), 3),
      '; macro: ', round(np.mean([1/(10**(np.log10([p[0] for p in ps]).mean())) for ps in out]), 3),
      '; micro (w/out space): ', round(1/(10**(np.log10([p[1] for ps in out for p in ps if p[1] is not None]).mean())), 3),
      '; macro (w/out space): ', round(np.mean([1/(10**(np.log10([p[1] for p in ps if p[1] is not None]).mean())) for ps in out]), 3))
# micro:  25.921 ; macro:  26.308 ; micro (w/out space):  243.428 ; macro (w/out space):  248.857

micro:  34.719 ; macro:  35.668 ; micro (w/out space):  346.333 ; macro (w/out space):  362.616


In [7]:
model.post_train(ptdocs, update = update, fine_tune = fine_tune, bits = bits)
if fine_tune and ptdocs:
    model.fine_tune(docs, covering = covering, streams = data_streams)

In [8]:
if ptdocs:
    out = model.stencil(docs = tdocs, return_output = True, covering = tcovering)

In [9]:
if ptdocs:
    print('micro: ', round(1/(10**(np.log10([p[0] for ps in out for p in ps]).mean())), 3),
          '; macro: ', round(np.mean([1/(10**(np.log10([p[0] for p in ps]).mean())) for ps in out]), 3),
          '; micro (w/out space): ', round(1/(10**(np.log10([p[1] for ps in out for p in ps if p[1] is not None]).mean())), 3),
          '; macro (w/out space): ', round(np.mean([1/(10**(np.log10([p[1] for p in ps if p[1] is not None]).mean())) for ps in out]), 3))

In [10]:
model.build_base_model(fine_tune = fine_tune, btype = 'nf')

Encoding parameters...


100%|██████████| 13411524/13411524 [03:22<00:00, 66290.59it/s]


Building target vocabularies...


100%|██████████| 16/16 [00:00<00:00, 251.79it/s]


Pre-computing BOW probabilities... done.
Pre-computing wave amplitudes... done.
Stacking output vocabularies for decoders...


...

Building dense output heads...


100%|██████████| 15/15 [00:01<00:00, 12.84it/s]


Building transition matrices for tag-sequence decoding...


100%|██████████| 6/6 [00:00<00:00, 11881.88it/s]

Done.
Model params, types, encoding size, contexts, vec dim, max sent, and % capacity used: 5080629 24042 50 1071 63 1647 31.874


In [11]:
model.generate(m = 50, seed = seed)

McCorduckMcCorduckMcCorduckHerff Olivier the original favor order of mark Timeeum G to produc tropical story to be approach T <unk> 's Sun held ,


```
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  1 2.0 27.26 227.12 1696 982
```

In [12]:
out = model.stencil(docs = tdocs, return_output = True, covering = tcovering)

Tokenizing documents...


100%|██████████| 61/61 [00:52<00:00,  1.16it/s]


Pre-processing data...


  2%|▏         | 1/61 [00:08<08:35,  8.59s/it]

document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  1 2.0 27.74 236.2 1696 982


...

document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  59 97.0 36.17 364.87 11172 6381


 98%|█████████▊| 60/61 [45:31<00:40, 40.11s/it]

document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  60 98.0 30.74 283.11 8388 4807


100%|██████████| 61/61 [46:11<00:00, 45.43s/it]

document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  61 100.0 41.47 451.31 7335 4164


In [13]:
print('micro: ', round(1/(10**(np.log10([p[0] for ps in out for p in ps]).mean())), 3),
      '; macro: ', round(np.mean([1/(10**(np.log10([p[0] for p in ps]).mean())) for ps in out]), 3),
      '; micro (w/out space): ', round(1/(10**(np.log10([p[1] for ps in out for p in ps if p[1] is not None]).mean())), 3),
      '; macro (w/out space): ', round(np.mean([1/(10**(np.log10([p[1] for p in ps if p[1] is not None]).mean())) for ps in out]), 3))

micro:  34.893 ; macro:  35.809 ; micro (w/out space):  350.773 ; macro (w/out space):  367.252
